# Custom Syft Workers

#### Setup 

In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'

In [ ]:
import syft as sy
import os
sy.requires(SYFT_VERSION)
from syft.service.worker.worker_image import SyftWorkerImage
from syft.service.worker.worker_pool import WorkerStatus, SyftWorker
from syft.custom_worker.config import DockerWorkerConfig
from syft.service.worker.image_registry import SyftImageRegistry

#### Launch Domain

In [ ]:
# Disable inmemory worker for container stack
in_memory_workers = not os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE") == "container_stack"

In [ ]:
domain = sy.orchestra.launch(name="test-domain-1", reset=True, dev_mode=True, create_producer=True, in_memory_workers=in_memory_workers)

In [ ]:
domain_client = domain.login(email="info@openmined.org", password="changethis")

#### Prepare Worker Image configuration

We first prepare a `DockerWorkerConfig` that can be a cog-like `dict`, a `Path` to a Dockerfile or just plain string!

In [ ]:
nginx_dockerfile_str = """
# Use the official Nginx image as the base
FROM openmined/grid-backend:0.8.4-beta.10

RUN pip install pydicom

"""

In [ ]:
docker_config = DockerWorkerConfig(dockerfile=nginx_dockerfile_str)
docker_config

In [ ]:
assert docker_config.dockerfile == nginx_dockerfile_str

#### Submit Worker Image config for review

In [ ]:
res = domain_client.api.services.worker_image.submit_dockerfile(docker_config=docker_config)
res

In [ ]:
assert isinstance( res, SyftWorkerImage)

In [ ]:
dockerfile_list = domain_client.api.services.worker_image.get_all()
dockerfile_list

In [ ]:
assert len(dockerfile_list) == 2

In [ ]:
workerimage = None
for image in dockerfile_list:
    if image.config.dockerfile == nginx_dockerfile_str:
        workerimage = image
        break

assert isinstance(workerimage, SyftWorkerImage)

#### Setup Registry

In [ ]:

import docker 

class TestRegistry:
    def __init__(self):
        self.name = 'local_registry'
        self.client = docker.from_env()
    
    def start(self, host_port=5678):
        try:
            existing = self.get()
            if existing:
                return existing

            result = self.client.containers.run("registry:2",
                                        name="local_registry",
                                        detach=True,
                                        ports={'5000/tcp': host_port})
            
            return result
        except Exception as e:
            print(e)
            return None

    def teardown(self):
        existing = self.get()
        if existing:
            existing.stop()
            existing.remove()

    def get(self):
        try:
            result = self.client.containers.get(self.name)
            if result.status == 'running':
                return result
        except docker.errors.NotFound:
            return None

local_registry_container = TestRegistry()
local_registry_container.start()

#### Add registry in Syft

In [ ]:
local_registry = domain_client.api.services.image_registry.add(url="localhost:5678")
local_registry

In [ ]:
assert isinstance(local_registry, SyftImageRegistry)

In [ ]:
registries = domain_client.api.services.image_registry.get_all()
registries

In [ ]:
assert len(registries) == 1

#### Build Worker Image

In [ ]:
docker_tag = "openmined/test-image"
docker_build_res = domain_client.api.services.worker_image.build(image=workerimage.id, tag=docker_tag, version="1", registry=local_registry.id)
docker_build_res

In [ ]:
print(docker_build_res.message)

In [ ]:
assert isinstance(docker_build_res, sy.SyftSuccess)

In [ ]:
docker_tag

In [ ]:
import subprocess

def check_image_exists(tag) -> bool:
    result = subprocess.run(['docker', 'images', '-q', tag], stdout=subprocess.PIPE)
    return result.stdout.strip() != b''

if domain.deployment_type.value == "container_stack":
    assert check_image_exists(docker_tag)

In [ ]:
image_list = domain_client.api.services.worker_image.get_all()
image_list

In [ ]:
for image in image_list:
    if image.id == workerimage.id:
        workerimage = image

In [ ]:
assert len(image_list) == 2

In [ ]:
import docker

def get_image_hash(tag) -> str:
    client = docker.from_env()
    try:
        image = client.images.get(tag)
        return image.id
    except docker.errors.ImageNotFound:
        return None

In [ ]:
if domain.deployment_type.value == "container_stack":
    assert workerimage.image_hash == get_image_hash(docker_tag)

In [ ]:
def get_container_id(container_name: str) -> str:
    client = docker.from_env()
    try:
        container = client.containers.get(container_name)
        return container.id
    except docker.errors.NotFound:
        return None

#### Push built image to repository

In [ ]:
push_result = domain_client.api.services.worker_image.push(image=workerimage.id)
push_result

In [ ]:
if domain.deployment_type.value == "container_stack":
    assert isinstance(push_result, sy.SyftSuccess)

In [ ]:
local_registry_container.teardown()

#### Create Syft Worker Pool

In [ ]:
worker_pool_name = "my_first_worker_pool"
worker_pool_res = domain_client.api.services.worker_pool.create(name=worker_pool_name, image_uid = workerimage.id, number=3)

In [ ]:
assert len(worker_pool_res) == 3

In [ ]:
for status in worker_pool_res:
    assert status.error == None
    if domain.deployment_type.value == "container_stack":
        assert status.worker.image_hash == get_image_hash(docker_tag)

In [ ]:
worker_pool_list = domain_client.api.services.worker_pool.get_all()

In [ ]:
assert len(worker_pool_list)==2
worker_pool = None
for pool in worker_pool_list:
    if pool.name == worker_pool_name:
        worker_pool = pool
        break
assert worker_pool is not None
assert len(worker_pool.workers)==3

In [ ]:
# Delete the second worker
second_worker = worker_pool.workers[1]

#### Delete a Syft Worker from a Pool

In [ ]:
second_worker

In [ ]:
raw_worker_logs = domain_client.api.services.worker_pool.worker_logs(
    worker_pool_id=worker_pool.id, worker_id=second_worker.id, raw=True
)
raw_worker_logs

In [ ]:
assert isinstance(raw_worker_logs, bytes)

In [ ]:
worker_logs = domain_client.api.services.worker_pool.worker_logs(
    worker_pool_id=worker_pool.id, worker_id=second_worker.id
)
worker_logs

In [ ]:
assert isinstance(worker_logs, str)

In [ ]:
worker_delete_res = domain_client.api.services.worker_pool.delete_worker(
    worker_pool_id=worker_pool.id, worker_id=second_worker.id
)

In [ ]:
worker_delete_res

In [ ]:
assert isinstance(worker_delete_res, sy.SyftSuccess)

In [ ]:
# Refetch the worker pool
# Ensure that the deleted worker's id is not present
for pool in domain_client.api.services.worker_pool.get_all():
    if pool.name == worker_pool_name:
        worker_pool = pool
assert len(worker_pool.workers) == 2
for worker in worker_pool.workers:
    assert second_worker.id != worker.id

#### Delete a Syft Worker Image

In [ ]:
# delete the remaining workers
for worker in worker_pool.workers:
    res =domain_client.api.services.worker_pool.delete_worker(
        worker_pool_id=worker_pool.id, worker_id= worker.id
    )
    assert isinstance(res, sy.SyftSuccess)

In [ ]:
delete_res = domain_client.api.services.worker_image.delete(workerimage.id)

In [ ]:
# Since the containers are delete, we should be able to delete the image
assert isinstance(delete_res, sy.SyftSuccess)
delete_res